In [ ]:
! pip install datasets transformers evaluate -q
! pip install tensorflow
! pip install torch torchvision
! pip install matplotlib
! pip install tf-keras
! pip install scikit-learn
! pip install tqdm

Python(82833) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C


Python(82835) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(82837) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(82839) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(82845) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python(82853) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user


In [20]:
! pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 19.6 MB/s eta 0:00:00


In [ ]:
! pip install git+https://github.com/casper-hansen/AutoAWQ.git@1c5ccc791fa2cb0697db3b4070df1813f1736208

In [ ]:
from awq import AutoAWQForCausalLM

model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=False)

text_generation = pipeline("text-generation", model=model)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [ ]:
from datasets import load_from_disk
from transformers import pipeline, AutoTokenizer
import tqdm

# Read the BOLD JSON files into a dataset
load_path='./prompts/sample_bold_prompts'
bold_prompts = load_from_disk(load_path)

# Load text generation pipeline and model
model = "mistralai/Mistral-7B-v0.1"
text_generation = pipeline("text-generation", model=model)
tokenizer = AutoTokenizer.from_pretrained(model)

Loading checkpoint shards: 100%|██████████| 2/2 [01:37<00:00, 48.52s/it]


In [ ]:
def generate_text(prompts, max_length=100, do_sample=False, pad_token_id=502256):
	model_continuations=[]
	# Iterate through prompts and generate text
	for prompt in tqdm.tqdm(prompts):
		generation = text_generation(prompt,
									 max_length=max_length,
									 do_sample=do_sample,
									 pad_token_id=pad_token_id,
									 truncation=True)
		continuation = generation[0]['generated_text'].replace(prompt,'').lstrip()
		model_continuations.append(continuation)
	return model_continuations

In [ ]:
# Select only first instance of prompt, some prompts have multiple sentences
toxic_prompts = [p['prompts'] for p in bold_prompts]

# Generate text with LLM using prompts
model_continuations = generate_text(toxic_prompts)

print('Generated '+ str(len(model_continuations))+ ' continuations')

# Update dataset with generations
bold_prompts = bold_prompts.add_column(f'generated_prompts_{model}', model_continuations)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [08:32<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Save resulting prompts to disk
out_name = f'./generated_data/prompts_{model}'
bold_prompts.save_to_disk(out_name)